<a href="https://colab.research.google.com/github/zydane07/ML-Dev-Yourney/blob/main/Capstone_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Library**

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import string

# **Read File**

In [4]:
df = pd.read_csv('Mental_Health_FAQ.csv')

In [5]:
df.drop('Question_ID', axis=1, inplace=True)

# **Split**

In [6]:
question = df['Questions'].tolist()
answer = df['Answers'].tolist()

# **LowerCase**

In [7]:
question = [q.lower() for q in question]
answer = [a.lower() for a in answer]

# **StopWords**

In [8]:
stopwords = []
with open('stopwords.txt', encoding='utf-8') as f:
  for line in f:
    stopwords.append(line.strip())

In [9]:
print(stopwords)

['a', 'about', 'above', 'across', 'after', 'again', 'against', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'among', 'an', 'and', 'another', 'any', 'anybody', 'anyone', 'anything', 'anywhere', 'are', 'area', 'areas', 'around', 'as', 'ask', 'asked', 'asking', 'asks', 'at', 'away', 'b', 'back', 'backed', 'backing', 'backs', 'be', 'became', 'because', 'become', 'becomes', 'been', 'before', 'began', 'behind', 'being', 'beings', 'best', 'better', 'between', 'big', 'both', 'but', 'by', 'c', 'came', 'can', 'cannot', 'case', 'cases', 'certain', 'certainly', 'clear', 'clearly', 'come', 'could', 'd', 'did', 'differ', 'different', 'differently', 'do', 'does', 'done', 'down', 'down', 'downed', 'downing', 'downs', 'during', 'e', 'each', 'early', 'either', 'end', 'ended', 'ending', 'ends', 'enough', 'even', 'evenly', 'ever', 'every', 'everybody', 'everyone', 'everything', 'everywhere', 'f', 'face', 'faces', 'fact', 'facts', 'far', 'felt', 'few', 'find', 'finds', 'first

# **Cleaning**

In [10]:
question_clean = []
for i in question:
  temp = [w for w in i.split() if w not in stopwords]
  temp = " ".join(temp)
  temp = re.sub(r'[.,’"\'-?:!;]', '', temp)
  temp = re.sub(r'^whats|^im', '', temp)
  question_clean.append(temp.strip())
question_clean

['mean mental illness',
 'mental illness affect',
 'causes mental illness',
 'warning signs mental illness',
 'people mental illness recover',
 'appears symptoms mental disorder',
 'mental health professional child',
 'treatment options available',
 'involved treatment know',
 'difference mental health professionals',
 'mental health professional child myself',
 'involved treatment know',
 'else help',
 'starting medication',
 'feel taking medication mean am cured stop taking it',
 'help paying medication',
 'therapy',
 'learn types mental health treatment',
 'types mental health professionals',
 'support group',
 'inpatient care',
 'local services',
 'learn information clinical trials',
 'learn information creating psychiatric advance directive',
 'mental health',
 'support doesnt help',
 'prevent mental health problems',
 'cures mental health problems',
 'causes mental health problems',
 'worried mental health',
 'unwell',
 'worried friend relative',
 'deal telling do',
 'informed',


In [11]:
answer_clean = []
for i in answer:
  temp = [w for w in i.split()]
  temp = " ".join(temp)
  temp = temp.encode("ascii", "ignore")
  answer_clean.append(temp.decode())
answer_clean

['mental illnesses are health conditions that disrupt a persons thoughts, emotions, relationships, and daily functioning. they are associated with distress and diminished capacity to engage in the ordinary activities of daily life. mental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. on the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. these include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital. it is important to know that mental illnesses are medical conditions that have nothing to do with a persons character, intelligence, or willpower. just as diabetes is a disorder of the pancreas, mental illness is a medical condition due to the brains biology. similarly to how one would treat diabetes with medication and insulin, men

In [12]:
def fit_tokenizer(train_sentences, num_words, oov_token, max_len):
    
    tokenizer = Tokenizer(num_words=num_words, oov_token=oov_token)

    
    tokenizer.fit_on_texts(train_sentences)
    sequences = tokenizer.texts_to_sequences(train_sentences)
    padd_seq = pad_sequences(sequences, truncating='post', maxlen=max_len)
    
    return padd_seq, tokenizer

In [13]:
train_seq, tokenizer = fit_tokenizer(question_clean, 1000, "<OOV>", 100)

In [14]:
lb = LabelEncoder()
val_label_seq = np.array(lb.fit_transform(answer_clean))


In [15]:
def create_model(vocab_size, embedding_dim, maxlen):
    
    model = tf.keras.Sequential([ 
        tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=maxlen),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(98, activation='softmax')
    ])
    
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy']) 


    return model

In [16]:
model = create_model(len(tokenizer.word_index), 16, 100)
# print(train_padded_seq.shape)
# print(train_label_seq.shape)

history = model.fit(train_seq, val_label_seq, epochs=200)

Epoch 1/200
4/4 [==============================] - 1s 8ms/step - loss: 4.5876 - accuracy: 0.0204
Epoch 2/200
4/4 [==============================] - 0s 7ms/step - loss: 4.5831 - accuracy: 0.0204
Epoch 3/200
4/4 [==============================] - 0s 6ms/step - loss: 4.5803 - accuracy: 0.0408
Epoch 4/200
4/4 [==============================] - 0s 7ms/step - loss: 4.5777 - accuracy: 0.0306
Epoch 5/200
4/4 [==============================] - 0s 9ms/step - loss: 4.5758 - accuracy: 0.0408
Epoch 6/200
4/4 [==============================] - 0s 6ms/step - loss: 4.5725 - accuracy: 0.0510
Epoch 7/200
4/4 [==============================] - 0s 7ms/step - loss: 4.5695 - accuracy: 0.0408
Epoch 8/200
4/4 [==============================] - 0s 7ms/step - loss: 4.5651 - accuracy: 0.0408
Epoch 9/200
4/4 [==============================] - 0s 9ms/step - loss: 4.5602 - accuracy: 0.0612
Epoch 10/200
4/4 [==============================] - 0s 7ms/step - loss: 4.5555 - accuracy: 0.0612
Epoch 11/200
4/4 [===========

In [17]:
a = input()

temp = [w for w in a.split() if w not in stopwords]
temp = " ".join(temp)
temp = re.sub(r'[.,’"\'-?:!;]', '', temp)
temp = re.sub(r'^whats|^im', '', temp)
temp = temp.strip()

result = model.predict(pad_sequences(tokenizer.texts_to_sequences([temp]),
                                             truncating='post', maxlen=100))
q = lb.inverse_transform([np.argmax(result)])
print(q)

support doesnt help
["it can be difficult to find the things that will help you, as different things help different people. it's important to be open to a range of approaches and to be committed to finding the right help and to continue to be hopeful, even when some things don't work out."]
